In [48]:
import numpy as np

In [49]:
n = 27
d=2
c=3
def sigmoid(x):
  return 1/(1+np.exp(-x))
X = np.ones(n*d).reshape(n,d)
W = np.ones(d*c).reshape(d,c)
B = np.ones(n*c).reshape(n,c) 
b = np.zeros(c)
print(W.shape,B.shape,X.shape)
K = sum(sigmoid(np.dot(X,W)+B))
print(np.repeat(b,n).reshape(n,c).shape)
def softmax(z):
  n,d = z.shape
  matrix_a = np.zeros(n*d).reshape(n,d)
  for i in range(n):
    denominator = 0
    for j in range(d):
      denominator += np.exp(z[i][j])
    for j in range(d):
      matrix_a[i][j] = np.exp(z[i][j]) / denominator
      
  return matrix_a

# print(softmax(np.array([[1,1,1],[1,1,0],[1,0,0],[2,3,5]])))
print((softmax(np.dot(X,W)+B)).sum())

(2, 3) (27, 3) (27, 2)
(27, 3)
27.0


In [65]:
            ###################### Cam NN #####################

def sigmoid(z):
  return 1/(1+np.exp(-z))

def sigmoid_derivative(z):
  return z*(1-z)

def softmax(z):
  n,d = z.shape
  matrix_a = np.zeros(n*d).reshape(n,d)
  for i in range(n):
    denominator = 0
    for j in range(d):
      denominator += np.exp(z[i][j])
    for j in range(d):
      matrix_a[i][j] = np.exp(z[i][j]) / denominator
      
  return matrix_a


class HiddenLayer:
  def __init__(self,X,output_nodes=5):
    self.alpha = 0.03                    # learning rate
    self.X = np.array(X)                 # shape: n*d
    self.n,self.d = X.shape
    self.c = output_nodes
    self.W = np.ones(self.d*self.c).reshape(self.d,self.c)          # shape: d*c
    self.b = np.zeros(self.c)            # shape: 1*c
    self.B = None
    self.A = None
    self.dL_dA = np.ones(self.n*self.c).reshape(self.n,self.c)      # shape: n*c
    self.dL_dX = None
  
  def updateX(self,X):
    self.X = np.array(X)   # shape: n*d
    self.n,self.d = X.shape
  
  def updateDL_DA(self,dl_da):
    self.dL_dA = dl_da
  
  def returnDL_DA(self):
    return self.dL_dA     # shape n*c
    
  def returnDL_DX(self):
    return self.dL_dX     # shape n*c

  def FeedForward(self):
    self.B = np.repeat(self.b,self.n).reshape(self.n,self.c)          # shape: n*c
    self.A = sigmoid(np.dot(self.X,self.W) + self.B)                  # shape: n*c
  
  def BackPropagte(self):
    dA_dZ = sigmoid_derivative(self.A)                                # shape: n*c
    db = sum(np.multiply(self.dL_dA,dA_dZ))                           # shape: 1*c
    dW = np.dot( self.X.transpose() , np.multiply(self.dL_dA,dA_dZ) ) # shape: d*c
    dX = np.dot( np.multiply(self.dL_dA,dA_dZ) , self.W.transpose() ) # shape: n*d    -> dl_da of previous layer
    self.W -= self.alpha * dW
    self.b -= self.alpha * db
    self.dL_dX = dX                                                   # pass to previous layer
  
  def returnMatrix(self,option):
    if option == 'a':
      return self.A
    elif option == 'w':
      return self.W
    elif option =='b':
      return self.b
    elif option =='x':
      return self.X
    
class LastHiddenLayer :
  def __init__(self,X,P):
    self.alpha = 0.03                    # learning rate
    self.X = np.array(X)                 # shape: n*d
    self.P = np.array(P) 
    self.n,self.d = X.shape
    self.c = P.shape[1] 
    self.W = np.ones(self.d*self.c).reshape(self.d,self.c)   # shape: d*c
    self.b = np.zeros(self.c)            # shape: 1*c
    self.B = None
    self.A = None
    self.dL_dX = None
    self.dL_dZ = None                    # shape: n*c
    self.P = np.array(P)                 # shape: n*c

  def updateX(self,X):
    self.X = np.array(X)                 # shape: n*d
    self.n,self.d = X.shape

  def returnDL_DX(self):
    return self.dL_dX     # shape n*c

  def FeedForward(self):
    self.B = np.repeat(self.b,self.n).reshape(self.n,self.c)          # shape: n*c
    self.A = softmax(np.dot(self.X,self.W) + self.B)                  # shape: n*c
  
  def BackPropagte(self):                              
    self.dL_dZ =  self.A - self.P                   # shape: n*c
    db = sum(self.dL_dZ)                            # shape: 1*c
    dW = np.dot( self.X.transpose() , self.dL_dZ )  # shape: d*c
    dX = np.dot( self.dL_dZ , self.W.transpose() )  # shape: n*d    -> dl_da of previous layer
    self.W -= self.alpha * dW
    self.b -= self.alpha * db
    self.dL_dX = dX                               # pass to previous layer

  def LossFunction(self):
    return - 1/self.n * (self.P * np.log(self.A)).sum()
  
  def returnMatrix(self,option):
    if option == 'a':
      return self.A
    elif option == 'w':
      return self.W
    elif option =='b':
      return self.b
    elif option =='x':
      return self.X

  



In [66]:
########  Initialize NN ################

X = np.array([[1,2],[3,4],[5,6],[7,8]])        # shape: 4*2 
P = np.array([[0,1],[1,0],[1,0],[1,0]])        # shape: 4*2 


#################################### 2 epoch only  #################################

######### InitNetWork and 1st FORWARD ###########

input_layer = HiddenLayer(X,output_nodes=4)        # 4*2  ->  4*4
input_layer.FeedForward()
# print(input_layer.returnMatrix('a'))

first_layer = HiddenLayer(input_layer.returnMatrix('a'),output_nodes=5)        # 4*4  ->  4*5
first_layer.FeedForward()
# print(first_layer.returnMatrix('a'))

second_layer = HiddenLayer(first_layer.returnMatrix('a'),output_nodes=2)       # 4*5  ->  4*2
second_layer.FeedForward()
# print(second_layer.returnMatrix('a'))

last_layer = LastHiddenLayer(second_layer.returnMatrix('a'),P)                  # 4*2  -> 4*2
last_layer.FeedForward()
print(last_layer.returnMatrix('a'))

print('initial loss: ',last_layer.LossFunction())

########## 1st BACKWARD ###########
last_layer.BackPropagte()
second_layer.updateDL_DA(last_layer.returnDL_DX())

second_layer.BackPropagte()
first_layer.updateDL_DA(second_layer.returnDL_DX())

first_layer.BackPropagte()
input_layer.updateDL_DA(first_layer.returnDL_DX())

input_layer.BackPropagte()

##### 2nd FORWARD and Calculate loss 2nd time ##########
input_layer.FeedForward()
# print(input_layer.returnMatrix('a'))
# print(input_layer.returnMatrix('b'))

first_layer.updateX(input_layer.returnMatrix('a'))
first_layer.FeedForward()
# print(first_layer.returnMatrix('a'))
# print(first_layer.returnMatrix('b'))

second_layer.updateX(first_layer.returnMatrix('a'))
second_layer.FeedForward()
# print(second_layer.returnMatrix('a'))
# print(second_layer.returnMatrix('b'))

last_layer.updateX(second_layer.returnMatrix('a'))
last_layer.FeedForward()
print(last_layer.returnMatrix('a'))
print('after 1 epoch loss: ',last_layer.LossFunction())
print('We can see that loss actually decrease !')

[[0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]]
initial loss:  0.6931471805599453
[[0.52952609 0.47047391]
 [0.52953002 0.47046998]
 [0.52953009 0.47046991]
 [0.52953009 0.47046991]]
after 1 epoch loss:  0.6653276949813814
We can see that loss actually decrease !
